In [1]:
import os
import sys
sys.path.insert(0, '../')
import time
import json
import pandas as pd

from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import Indexer, Searcher

In [15]:
def read_df(pth_models, pth_offers):

    df_models = pd.read_csv(pth_models, sep=";")
    df_models = df_models.drop(columns=['average_price', 'comment'])
    df_offers = pd.read_csv(pth_offers, sep=";")
    df_offers = df_offers.drop(columns=['true_match', 'false_match'])

    return df_models, df_offers

pth_models = "/mnt/vdb1/Datasets/ColBERT/18_categories/test/models_18_categories.csv"
pth_offers = "/mnt/vdb1/Datasets/ColBERT/18_categories/test/triplets_test_18_categories.csv"

df_models, df_offers = read_df(pth_models, pth_offers)

df_offers

/tmp/ipykernel_2668/2656002276.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_models = pd.read_csv(pth_models, sep=";")


,name,model_id,category_name,category_id
0,Портативная акустика Hoco HC2 Xpress темно-зел...,4771613,портативная акустика,3904
1,Акустическая система HYUNDAI Колонка порт. H-P...,4570006,портативная акустика,3904
2,Кухонная мойка из искусственного камня Vigro V...,5769579,кухонные мойки,740101
3,Хиллс 604304 Puppy Large сух.д/щенков крупных ...,4386018,корм для собак,921401
4,"Смартфон Xiaomi POCO M5 4/128 GB Global, серый",5506115,мобильные телефоны,2801
...,...,...,...,...
135891,Кухонная Мойка Zorg X-78-2-44,3601922,кухонные мойки,740101
135892,"Balenciaga, Cristobal Pour Homme, 100 мл., туа...",3081853,парфюмерия,911906
135893,"Умная колонка MAIL Капсула, белый [mrc01(white)]",4855069,портативная акустика,3904
135894,Наушники беспроводные JBL Tune 750BTNC черные ...,4455835,"наушники, гарнитуры, наушники c микрофоном",2102


In [3]:
# ans = [y for x, y in df_offers.groupby('category_id')]
# thresh_upper = 10
# for i in range(len(ans)):
#     #ans[i] = ans[i].drop_duplicates(subset='name', keep="last")
#     ans[i] = ans[i].sample(frac=1).sample(frac=1)[:thresh_upper]
# df_new = pd.concat(ans, axis=0)
# df_new

,name,model_id,category_name,category_id
95201,Honor Беспроводные наушники Choice CE79 TWS Ea...,4564207,"наушники, гарнитуры, наушники c микрофоном",2102
85579,Наушники беспроводные SMARTBUY SBH-001 I7 TWS ...,4533334,"наушники, гарнитуры, наушники c микрофоном",2102
63579,Студийные наушники BEHRINGER HC 2000B,4731639,"наушники, гарнитуры, наушники c микрофоном",2102
65798,Беспроводные наушники Edifier W600BT (Grey),4783655,"наушники, гарнитуры, наушники c микрофоном",2102
78081,"Беспроводные наушники TWS i18, white (белый)",4802467,"наушники, гарнитуры, наушники c микрофоном",2102
...,...,...,...,...
117851,FRAIS CLASSIQUE ADULT DOG BEEF (сухой корм для...,4882472,корм для собак,921401
48513,"Сухой корм для собак Nature's Table , индейка,...",4900326,корм для собак,921401
19529,Корм сухой для собак Корм сухой Chicopee Pro N...,6277381,корм для собак,921401
9078,Сухой корм для щенков Meglium курица 1 шт. х 1...,4388423,корм для собак,921401


In [17]:
df_models

,model_id,name,brand_name,full_name,category_name,category_id
0,623742,920-005619,Logitech,Logitech 920-005619,"чехлы, обложки для гаджетов (телефонов, планше...",3994
1,721952,Zipper Bag,Hama,Hama Zipper Bag,"чехлы, обложки для гаджетов (телефонов, планше...",3994
2,721970,CC-3064,Nokia,Nokia CC-3064,"чехлы, обложки для гаджетов (телефонов, планше...",3994
3,751488,CKS-X7/R,Sony,Sony CKS-X7/R,"чехлы, обложки для гаджетов (телефонов, планше...",3994
4,751989,EP-031023,Era Pro,Era Pro EP-031023,"чехлы, обложки для гаджетов (телефонов, планше...",3994
...,...,...,...,...,...,...
103209,7049424,MD-108,Mivo,Mivo MD-108,портативная акустика,3904
103210,7049425,MD-165,Mivo,Mivo MD-165,портативная акустика,3904
103211,7049426,Boost 20W,Rocket,Rocket Boost 20W,портативная акустика,3904
103212,7049427,Motion 10W,Rocket,Rocket Motion 10W,портативная акустика,3904


In [5]:
categories = [
    "диктофоны, портативные рекордеры",
    "электронные книги",
    "автомобильные телевизоры, мониторы",
    "смарт-часы и браслеты",
    "портативные медиаплееры",
    "чехлы, обложки для гаджетов (телефонов, планшетов etc)",
    "портативная акустика",
    "мобильные телефоны",
    "VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)",
    "планшетные компьютеры и мини-планшеты",
    "наушники, гарнитуры, наушники c микрофоном",
    "радиоприемники, радиобудильники, радиочасы",
    "магнитолы",
    "GPS-навигаторы"
    ]

def search(checkpoint, offers, models, nbits, doc_maxlen):
    index_name = f'models.18_categories.{nbits}bits'

    offers = Queries(path=offers)
    models = Collection(path=models)

    start_time = time.time()
    with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use.
        config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits)

        indexer = Indexer(checkpoint=checkpoint, config=config)
        indexer.index(name=index_name, collection=models, overwrite=True)
    indexer.get_index() # You can get the absolute path of the index, if needed.

    with Run().context(RunConfig(experiment='notebook')):
        searcher = Searcher(index=index_name)
    with open("/mnt/vdb1/ColBERT/tmp/logs.txt", "a") as txt:
        txt.write(f"Подготовка моделей категории: time_spent = {time.time() - start_time}\n")
    print(f"Подготовка моделей категории: time_spent = {time.time() - start_time}\n")

    start_time = time.time()
    rankings = searcher.search_all(offers, k=5).todict()
    with open("/mnt/vdb1/ColBERT/tmp/logs.txt", "a") as txt:
        txt.write(f"Инференс на всех офферах категории: time_spent = {time.time() - start_time}\n")
    print(f"Инференс на всех офферах категории: time_spent = {time.time() - start_time}\n\n\n")
    return rankings

def ranking_index(rankings, category_rankings, df, index_of_first):
    """
    сделать сдвиг passage_id на величину index_of_first для приведения к формату в котором поиск оффера ведется среди моделей всех категорий 
    упорядочить (passage_id, rank, score) в rankings согласно изначальным индексам в df_offers
    """

    for i in category_rankings:
        for j in range(5):
            category_rankings[i][j] = (category_rankings[i][j][0] + index_of_first, category_rankings[i][j][1], category_rankings[i][j][2])
            
    i = -1
    for index, row in df.iterrows():
        i += 1
        rankings[index] = category_rankings[i]
    return rankings

def df_split(df, col="name"):
    df = df.reset_index(drop=True)

    df1 = pd.DataFrame()
    df1["id"], df1[col] = [i for i in range(len(df))], df[col]
    
    df2 = pd.DataFrame()
    df2["id"], df2["model_id"] = [i for i in range(len(df))], df['model_id']
    return df1, df2

def prepare_tsv(category_offers, category_models, pth_offers, pth_models):
    query, query_id = df_split(category_offers, col="name")
    query.to_csv(pth_offers, sep='\t', header=False, index=False)

    document, document_id = df_split(category_models, col="full_name")
    document.to_csv(pth_models, sep='\t', header=False, index=False)
    
tmp_fld = "/mnt/vdb1/ColBERT/tmp"
ckpt_pth = "/mnt/vdb1/ColBERT/experiments/HYPERPARAM_shuffle/none/2024-01/10/13.34.30/checkpoints/colbert-9555"
pth_dst_json = "/mnt/vdb1/Datasets/ColBERT/18_categories/metrics_data/EVAL/categories_one_by_one/triples_shuffle_colbert-9555.json"
doc_maxlen = 300
nbits = 2   # encode each dimension with 2 bits
rankings = {}
for category in categories:
    with open("/mnt/vdb1/ColBERT/tmp/logs.txt", "a") as txt:
        txt.write(f"{category}:\n")
    print(category)
    index_of_first = df_models.index[df_models['category_name'] == category].tolist()[0]

    category_models = df_models[df_models['category_name'] == category]
    category_offers = df_offers[df_offers['category_name'] == category]

    pth_models = f'{tmp_fld}/models.tsv'
    pth_offers = f'{tmp_fld}/offers.tsv'
    prepare_tsv(category_offers, category_models, pth_offers, pth_models)

    category_rankings = search(ckpt_pth, pth_offers, pth_models, nbits, doc_maxlen)
    rankings = ranking_index(rankings, category_rankings, category_offers, index_of_first)

with open(pth_dst_json, 'w') as fp:
    json.dump(rankings, fp)

with open("/mnt/vdb1/ColBERT/tmp/logs.txt", "a") as txt:
    txt.write(f"Save pth: {pth_dst_json}\n")

диктофоны, портативные рекордеры
[Jan 11, 16:26:23] #> Loading the queries from /mnt/vdb1/ColBERT/tmp/offers.tsv ...
[Jan 11, 16:26:23] #> Got 10 queries. All QIDs are unique.

[Jan 11, 16:26:23] #> Loading collection...
0M 


[Jan 11, 16:26:23] #> Note: Output directory /mnt/vdb1/ColBERT/experiments/notebook/indexes/models.18_categories.2bits already exists


[Jan 11, 16:26:23] #> Will delete 10 files already at /mnt/vdb1/ColBERT/experiments/notebook/indexes/models.18_categories.2bits in 20 seconds...


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "nbits": 2,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 9555,
    "save_every": null,
    "warmup": 0,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 300,
    "mask_punctuation": true,
    "checkpoint": "\/mnt\/vdb1\/ColBERT\/experiments\/HYPERPARAM_shuffle\/none\/2024-01\/10\/13.34.30\/checkpoints\/colbert-9555",


WARNING clustering 4558 points to 1024 centroids: please provide at least 39936 training points


[Jan 11, 16:26:52] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[0.027, 0.03, 0.025, 0.026, 0.023, 0.027, 0.021, 0.023, 0.027, 0.023, 0.022, 0.026, 0.025, 0.027, 0.022, 0.024, 0.025, 0.024, 0.021, 0.027, 0.024, 0.024, 0.027, 0.029, 0.023, 0.024, 0.023, 0.026, 0.024, 0.024, 0.024, 0.023, 0.022, 0.026, 0.029, 0.022, 0.027, 0.029, 0.028, 0.03, 0.023, 0.027, 0.023, 0.025, 0.025, 0.027, 0.026, 0.028, 0.023, 0.026, 0.029, 0.029, 0.024, 0.027, 0.025, 0.024, 0.023, 0.026, 0.027, 0.024, 0.026, 0.021, 0.025, 0.026, 0.024, 0.024, 0.025, 0.025, 0.024, 0.029, 0.023, 0.024, 0.029, 0.028, 0.03, 0.026, 0.023, 0.025, 0.028, 0.025, 0.028, 0.023, 0.027, 0.024, 0.026, 0.023, 0.029, 0.025, 0.029, 0.022, 0.019, 0.023, 0.023, 0.021, 0.024, 0.027, 0.023, 0.025, 0.025, 0.025, 0.024, 0.02, 0.021, 0.029, 0.029, 0.024, 0.026, 0.025, 0.027, 0.026, 0.025, 0.026, 0.024, 0.023, 0.023, 0.023, 0.027, 0.031, 0.025, 0.026, 0.028, 0.025, 0.028, 0.026, 0.023, 0.025, 0.026,

1it [00:00,  5.72it/s]
100%|██████████| 1024/1024 [00:00<00:00, 93948.89it/s]


...

[Jan 11, 16:35:21] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 549.78it/s]



#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Навигатор Garmin gpsmap 66st, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  1192, 10260, 25529, 10325, 29741, 10260, 22919, 14150,
        16856, 11721, 27512, 14658,  2863,  2361,  5764,  3367,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



10it [00:00, 123.41it/s]

time_spent = 0.14594554901123047



In [6]:
rankings

{104561: [(57717, 1, 26.078125),
  (57758, 2, 17.015625),
  (57747, 3, 15.8828125),
  (57672, 4, 15.609375),
  (57849, 5, 15.359375)],
 131559: [(57699, 1, 18.609375),
  (57737, 2, 15.6015625),
  (57698, 3, 15.46875),
  (57662, 4, 15.3671875),
  (57569, 5, 14.6640625)],
 113853: [(57797, 1, 23.515625),
  (57824, 2, 18.578125),
  (57745, 3, 18.1875),
  (57463, 4, 18.171875),
  (57716, 5, 18.109375)],
 63102: [(57705, 1, 27.671875),
  (57703, 2, 22.265625),
  (57706, 3, 21.765625),
  (57406, 4, 21.75),
  (57690, 5, 18.78125)],
 118019: [(57510, 1, 25.53125),
  (57812, 2, 21.046875),
  (57470, 3, 20.78125),
  (57471, 4, 20.546875),
  (57511, 5, 20.078125)],
 10140: [(57549, 1, 20.484375),
  (57619, 2, 19.578125),
  (57597, 3, 15.640625),
  (57685, 4, 15.4765625),
  (57795, 5, 12.96875)],
 101021: [(57753, 1, 29.53125),
  (57703, 2, 23.53125),
  (57408, 3, 22.359375),
  (57752, 4, 20.515625),
  (57692, 5, 20.46875)],
 64464: [(57844, 1, 23.84375),
  (57845, 2, 17.515625),
  (57852, 3, 14.1